### Práctica de Acceso a bases de datos

In [8]:

f_nac = "98-12-11"
aux = f_nac
aux = aux.split("-")
print(aux)
if len(aux) != 3 or len(aux[0]) != 4 or len(aux[1]) != 2 or len(aux[2]) != 2:
  print("La fecha debe seguir el siguiente formato: 'AAAA-MM-DD'")

['98', '12', '11']
La fecha debe seguir el siguiente formato: 'AAAA-MM-DD'


Considerar una base de datos acerca de la liga de fútbol formada por las siguientes tablas:

__Tabla "Equipos"__:

* Equipo Cadena (Valor único)
* Ciudad Cadena	
* Entrenador Cadena	
* Estadio Cadena	
* Aforo Entero	
* Marca Cadena	
* Patrocinador Cadena


__Tabla "Partidos"__:

* Registro Entero (Valor único)
* Jornada Entero
* Fecha Cadena
* Equipo1 (Hace referencia al campo Equipo de la tabla Equipos)
* Goles_Equipo1 Entero
* Equipo 2 (Hace referencia al campo Equipo de la tabla Equipos)
* Goles_Equipo2 Entero

Los datos de las tablas se encuentran en los archivos csv "Equipos.csv" y "Partidos.csv".

Se pide:
1.  Hacer una función que cree la base de datos "futbol" con sus tablas y datos correspondientes. Para ello habrá que procesar los archivos csv proporcionados. Por ejemplo habrá que separar los resultados de los partidos que aparecen en un formato x-y, de manera que se obtengan los valores x e y para guardarlos en la tabla Partidos. (3 puntos)
2.  Resolver las siguientes consultas(7 puntos):

* Nº total de partidos jugados y ganados y nombre del equipo ordenado decrecientemente por el nº de partidos jugados y ganados.

* Partidos jugados: nombre del equipo1, nombre del equipo2, resultado equipo1, resultado equipo2, estadio  ordenados por el nombre del equipo1

* Un derbi es un partido entre dos equipos de la misma ciudad. Se pide obtener los derbis jugados: nombre del equipo1, nombre del equipo2, resultado equipo1, resultado equipo2, ciudad, estadio  ordenados por el nombre del equipo1

* El número de la jornada y los goles marcados en cada jornada ordenados decrecientemente por los goles marcados.

* Partido/s con mayor número de goles: nombre del equipo1,resultado equipo1, nombre del equipo2, resultado equipo2

* Partido/s con mayor diferencia de goles: nombre del equipo1,resultado equipo1, nombre del equipo2, resultado equipo2
  
* Hallar la media de la puntuación de cada equipo y nombre del equipo ordenada decrecientemente por la media de la puntuación.


#### Normas de entrega

* Fecha tope de entrega: 14/10/2021
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre BasesDatos_GrupoX donde X será el número de grupo correspondiente.


__CREAR LA BASE DE DATOS__

In [ ]:
import sqlite3
import csv

# Crear la BD futbol y tablas correspondientes
conn=sqlite3.connect("futbol.sqlite3")
cur=conn.cursor()
cur.execute("DROP TABLE IF EXISTS Equipos")
cur.execute("DROP TABLE IF EXISTS Partidos")
cur.execute("CREATE TABLE Equipos (nombre TEXT PRIMARY KEY, ciudad TEXT, entrenador TEXT, estadio TEXT, aforo INTEGER, marca TEXT, patrocinador TEXT)")
cur.execute("CREATE TABLE Partidos (registro INTEGER PRIMARY KEY, jornada INTEGER, fecha TEXT, equipo1 TEXT, goles1 INTEGER, equipo2 TEXT, goles2 INTEGER, FOREIGN KEY(equipo1) REFERENCES Equipos(nombre), FOREIGN KEY(equipo2) REFERENCES Equipos(nombre))")
# close y commit
cur.close()
conn.commit()

In [ ]:
# Leer datos del fichero y cargarlos en la tabla Equipos
archivoEjemplo=open("Equipos.csv",encoding="utf8",errors='ignore')
ejemploLector=csv.reader(archivoEjemplo, delimiter = ";")
ejemploDatos=list(ejemploLector)

cur=conn.cursor()

# Insertar los datos en la tabla
for i in range(len(ejemploDatos)):
    nom = ejemploDatos[i][0]
    ciu = ejemploDatos[i][1]
    entre = ejemploDatos[i][2]
    estad = ejemploDatos[i][3]
    afor = ejemploDatos[i][4]
    marc = ejemploDatos[i][5]
    patro = ejemploDatos[i][6]
    cur.execute("INSERT INTO Equipos (nombre, ciudad, entrenador, estadio, aforo, marca, patrocinador) VALUES (?,?,?,?,?,?,?)", (nom,ciu,entre,estad,afor,marc,patro))

cur.close()
conn.commit()

In [ ]:
# Comprobar tabla Equipos
cur=conn.cursor()
cur.execute("SELECT * FROM Equipos")

while True:
    fila=cur.fetchone()
    if not fila: break
    print (fila)

cur.close()
conn.commit()

('Athletic Club Bilbao', 'Bilbao', 'Ernesto Valverde', 'San Mams', 53289, 'Nike', 'Petronor')
('Atletico Madrid', 'Madrid', 'Diego Simeone', 'Vicente Caldern', 54851, 'Nike', 'Azerbaiyn')
('CA Osasuna', 'Pamplona', 'Javi Gracia', 'El Sadar', 19553, 'Adidas?', 'Lacturale')
('Elche CF', 'Elche', 'Fran Escrib', 'Manuel Martnez Valero', 36017, 'Acerbis', 'Gioseppo?')
('FC Barcelona', 'Barcelona', 'Gerardo Martino', 'Camp Nou', 99354, 'Nike', 'Qatar Airways?')
('Getafe CF', 'Getafe', 'Cosmin Contra', 'Coliseum Alfonso Prez', 17393, 'Joma', 'Confremar')
('Granada CF', 'Granada', 'Lucas Alcaraz', 'Nuevo Los Crmenes', 22524, 'Luanvi', 'Por definir')
('Levante UD', 'Valencia', 'Joaqun Caparrs', 'Ciudad de Valencia', 25354, 'Kelme?', 'Comunitat Valenciana')
('Malaga CF', 'Malaga', 'Bernd Schuster', 'La Rosaleda', 30044, 'Nike', 'Unesco')
('Rayo Vallecano', 'Madrid', 'Paco Jmez', 'Campo de Ftbol de Vallecas', 14790, 'Erre', 'Por definir')
('Real Betis', 'Sevilla', 'Gabriel Caldern', 'Benito Villa

In [ ]:
# Leer el archivo csv y cargar los datos en la tabla partidos
archivoEjemplo2=open("Partidos.csv",encoding="utf8",errors='ignore')
ejemploLector2=csv.reader(archivoEjemplo2, delimiter = ",")
ejemploDatos2=list(ejemploLector2)

# Crear una segunda lista final donde se almacena la información al completo que se va a usar en la BD
resultados_list = list()
for i in range(1,len(ejemploDatos2)):
    resultado = ejemploDatos2[i][3].split(sep='-')  # resultado del partido array de dos posiciones una para los goles de cada equipo
    resultados_list.append([i, ejemploDatos2[i][0], ejemploDatos2[i][1], ejemploDatos2[i][2], resultado[0], ejemploDatos2[i][4], resultado[1]])

# print(resultados_list)

cur=conn.cursor()
cur.executemany("INSERT INTO Partidos (registro, jornada, fecha, equipo1, goles1, equipo2, goles2) values(?,?,?,?,?,?,?)", resultados_list)

cur.close()
conn.commit()

In [ ]:
# Comprobar tabla Partidos
cur=conn.cursor()
cur.execute("SELECT * FROM Partidos")

while True:
    fila=cur.fetchone()
    if not fila: break
    print (fila)

cur.close()
conn.commit()

(1, 1, 'Sat Aug 17 2013', 'Real Sociedad', 2, 'Getafe CF', 0)
(2, 1, 'Sat Aug 17 2013', 'Real Valladolid CF', 1, 'Athletic Club Bilbao', 2)
(3, 1, 'Sat Aug 17 2013', 'Valencia CF', 1, 'Malaga CF', 0)
(4, 1, 'Sun Aug 18 2013', 'Sevilla FC', 1, 'Atletico Madrid', 3)
(5, 1, 'Sun Aug 18 2013', 'CA Osasuna', 1, 'Granada CF', 2)
(6, 1, 'Sun Aug 18 2013', 'FC Barcelona', 7, 'Levante UD', 0)
(7, 1, 'Sun Aug 18 2013', 'Real Madrid', 2, 'Real Betis', 1)
(8, 1, 'Mon Aug 19 2013', 'UD Almeria', 2, 'Villarreal CF', 3)
(9, 1, 'Mon Aug 19 2013', 'Rayo Vallecano', 3, 'Elche CF', 0)
(10, 1, 'Mon Aug 19 2013', 'RC Celta Vigo', 2, 'RCD Espanyol', 2)
(11, 2, 'Fri Aug 23 2013', 'Getafe CF', 2, 'UD Almeria', 2)
(12, 2, 'Fri Aug 23 2013', 'Athletic Club Bilbao', 2, 'CA Osasuna', 0)
(13, 2, 'Sat Aug 24 2013', 'Villarreal CF', 2, 'Real Valladolid CF', 1)
(14, 2, 'Sat Aug 24 2013', 'Elche CF', 1, 'Real Sociedad', 1)
(15, 2, 'Sat Aug 24 2013', 'RCD Espanyol', 3, 'Valencia CF', 1)
(16, 2, 'Sun Aug 25 2013', 'Leva

### Consultas

Nº total de partidos jugados y ganados y nombre del equipo ordenado decrecientemente por el nº de partidos jugados y ganados.

In [ ]:
cur=conn.cursor()

cur.execute("SELECT nombre, sum(jornadas), sum(goles) FROM (SELECT nombre, jornadas, goles FROM (SELECT nombre, COUNT(nombre) AS jornadas, sum(Partidos.goles1) AS goles from Equipos join Partidos on Partidos.equipo1=Equipos.nombre GROUP BY nombre ORDER BY goles DESC) UNION SELECT nombre, jornadas, goles FROM (SELECT nombre, COUNT(nombre) AS jornadas, sum(Partidos.goles2) AS goles from Equipos join Partidos on Partidos.equipo2=Equipos.nombre GROUP BY nombre ORDER BY goles DESC))GROUP BY nombre ORDER BY goles DESC")

while True:
    fila=cur.fetchone()
    if not fila: break
    print (fila)
    
cur.close()
conn.commit()

('FC Barcelona', 38, 100)
('Real Madrid', 38, 104)
('Atletico Madrid', 38, 77)
('Athletic Club Bilbao', 38, 66)
('Sevilla FC', 38, 69)
('Real Sociedad', 38, 62)
('Villarreal CF', 38, 60)
('Valencia CF', 38, 51)
('Rayo Vallecano', 38, 46)
('RC Celta Vigo', 38, 49)
('UD Almeria', 38, 43)
('Malaga CF', 38, 39)
('RCD Espanyol', 38, 41)
('CA Osasuna', 38, 32)
('Getafe CF', 38, 35)
('Real Valladolid CF', 38, 38)
('Levante UD', 38, 35)
('Real Betis', 38, 36)
('Granada CF', 38, 32)
('Elche CF', 38, 30)


Partidos jugados: nombre del equipo1, nombre del equipo2, resultado equipo1, resultado equipo2, estadio  ordenados por el nombre del equipo1

In [ ]:
cur=conn.cursor()

cur.execute("")

while True:
    fila=cur.fetchone()
    if not fila: break
    print (fila)
    
cur.close()
conn.commit()

Un derbi es un partido entre dos equipos de la misma ciudad. Se pide obtener los derbis jugados: nombre del equipo1, nombre del equipo2, resultado equipo1, resultado equipo2, ciudad, estadio  ordenados por el nombre del equipo1

In [ ]:
cur=conn.cursor()

cur.execute("SELECT a.nombre, a.equipo2, a.goles1, a.goles2, a.ciudad, a.estadio FROM (SELECT nombre, Partidos.equipo2, Partidos.goles1, Partidos.goles2, ciudad, estadio FROM Equipos JOIN Partidos ON nombre=Partidos.equipo1) a, Equipos WHERE (Equipos.ciudad=a.ciudad AND Equipos.nombre=a.equipo2) ORDER BY a.nombre")

while True:
    fila=cur.fetchone()
    if not fila: break
    print (fila)
    
cur.close()
conn.commit()

('Atletico Madrid', 'Rayo Vallecano', 5, 0, 'Madrid', 'Vicente Caldern')
('Atletico Madrid', 'Real Madrid', 2, 2, 'Madrid', 'Vicente Caldern')
('FC Barcelona', 'RCD Espanyol', 1, 0, 'Barcelona', 'Camp Nou')
('Levante UD', 'Valencia CF', 2, 0, 'Valencia', 'Ciudad de Valencia')
('RCD Espanyol', 'FC Barcelona', 0, 1, 'Barcelona', 'Cornell-El Prat')
('Rayo Vallecano', 'Real Madrid', 2, 3, 'Madrid', 'Campo de Ftbol de Vallecas')
('Rayo Vallecano', 'Atletico Madrid', 2, 4, 'Madrid', 'Campo de Ftbol de Vallecas')
('Real Betis', 'Sevilla FC', 0, 2, 'Sevilla', 'Benito Villamarn')
('Real Madrid', 'Atletico Madrid', 0, 1, 'Madrid', 'Estadio Santiago Bernabu')
('Real Madrid', 'Rayo Vallecano', 5, 0, 'Madrid', 'Estadio Santiago Bernabu')
('Sevilla FC', 'Real Betis', 4, 0, 'Sevilla', 'Ramn Snchez Pizjuan')
('Valencia CF', 'Levante UD', 2, 0, 'Valencia', 'Mestalla')


El número de la jornada y los goles marcados en cada jornada ordenados decrecientemente por los goles marcados.

In [ ]:
cur=conn.cursor()
#Partidos (registro, jornada, fecha, equipo1, goles1, equipo2, goles2)
cur.execute("SELECT jornada, SUM(Partidos.goles1+Partidos.goles2) AS goles FROM Partidos GROUP BY jornada ORDER BY goles DESC")

while True:
    fila=cur.fetchone()
    if not fila: break
    print (fila)
    
cur.close()
conn.commit()

(14, 42)
(21, 38)
(17, 37)
(1, 35)
(13, 33)
(11, 33)
(4, 33)
(10, 32)
(23, 30)
(8, 30)
(35, 29)
(32, 29)
(6, 29)
(3, 29)
(34, 28)
(29, 28)
(28, 28)
(19, 28)
(27, 27)
(16, 27)
(15, 27)
(5, 27)
(22, 26)
(18, 26)
(2, 25)
(38, 24)
(37, 24)
(36, 24)
(33, 24)
(31, 24)
(20, 24)
(25, 23)
(12, 23)
(26, 22)
(30, 21)
(24, 20)
(9, 19)
(7, 17)


Partido/s con mayor número de goles: nombre del equipo1,resultado equipo1, nombre del equipo2, resultado equipo2

In [ ]:
cur=conn.cursor()

cur.execute("SELECT (goles1+goles2) AS goles, equipo1, goles1, equipo2, goles2 from Partidos WHERE goles > 0 ORDER BY GOLES DESC")

while True:
    fila=cur.fetchone()
    if not fila: break
    print (fila)
    
cur.close()
conn.commit()

(10, 'Real Madrid', 7, 'Sevilla FC', 3)
(7, 'FC Barcelona', 7, 'Levante UD', 0)
(7, 'Atletico Madrid', 7, 'Getafe CF', 0)
(7, 'Real Sociedad', 4, 'RC Celta Vigo', 3)
(7, 'Getafe CF', 2, 'FC Barcelona', 5)
(7, 'Rayo Vallecano', 2, 'Villarreal CF', 5)
(7, 'Athletic Club Bilbao', 6, 'UD Almeria', 1)
(7, 'FC Barcelona', 7, 'CA Osasuna', 0)
(7, 'Real Madrid', 3, 'FC Barcelona', 4)
(7, 'UD Almeria', 4, 'Real Sociedad', 3)
(7, 'Real Betis', 4, 'Real Valladolid CF', 3)
(6, 'Atletico Madrid', 4, 'UD Almeria', 2)
(6, 'Real Madrid', 5, 'Real Sociedad', 1)
(6, 'Real Sociedad', 5, 'Real Betis', 1)
(6, 'RCD Espanyol', 4, 'Real Valladolid CF', 2)
(6, 'Villarreal CF', 5, 'Real Sociedad', 1)
(6, 'Athletic Club Bilbao', 4, 'Real Valladolid CF', 2)
(6, 'RC Celta Vigo', 4, 'Real Betis', 2)
(6, 'CA Osasuna', 1, 'Athletic Club Bilbao', 5)
(6, 'Rayo Vallecano', 2, 'Atletico Madrid', 4)
(6, 'Real Madrid', 4, 'Villarreal CF', 2)
(6, 'FC Barcelona', 6, 'Rayo Vallecano', 0)
(6, 'Getafe CF', 3, 'Granada CF', 3)
(

Partido/s con mayor diferencia de goles: nombre del equipo1,resultado equipo1, nombre del equipo2, resultado equipo2


In [ ]:
cur=conn.cursor()

cur.execute("SELECT (abs(goles1-goles2)) AS goles, equipo1, goles1, equipo2, goles2 from Partidos ORDER BY GOLES DESC")

while True:
    fila=cur.fetchone()
    if not fila: break
    print (fila)
    
cur.close()
conn.commit()

(7, 'FC Barcelona', 7, 'Levante UD', 0)
(7, 'Atletico Madrid', 7, 'Getafe CF', 0)
(7, 'FC Barcelona', 7, 'CA Osasuna', 0)
(6, 'FC Barcelona', 6, 'Rayo Vallecano', 0)
(5, 'Atletico Madrid', 5, 'Rayo Vallecano', 0)
(5, 'Malaga CF', 5, 'Rayo Vallecano', 0)
(5, 'Malaga CF', 0, 'RC Celta Vigo', 5)
(5, 'Atletico Madrid', 5, 'Real Betis', 0)
(5, 'Real Sociedad', 5, 'CA Osasuna', 0)
(5, 'UD Almeria', 0, 'Real Madrid', 5)
(5, 'Athletic Club Bilbao', 6, 'UD Almeria', 1)
(5, 'Real Betis', 0, 'Real Madrid', 5)
(5, 'Valencia CF', 5, 'Real Betis', 0)
(5, 'Real Madrid', 5, 'Rayo Vallecano', 0)
(4, 'Rayo Vallecano', 0, 'FC Barcelona', 4)
(4, 'Real Madrid', 7, 'Sevilla FC', 3)
(4, 'Real Madrid', 5, 'Real Sociedad', 1)
(4, 'FC Barcelona', 4, 'Granada CF', 0)
(4, 'Sevilla FC', 4, 'Real Betis', 0)
(4, 'Real Madrid', 4, 'Real Valladolid CF', 0)
(4, 'Real Sociedad', 5, 'Real Betis', 1)
(4, 'FC Barcelona', 4, 'Elche CF', 0)
(4, 'Granada CF', 4, 'Real Valladolid CF', 0)
(4, 'Villarreal CF', 5, 'Real Sociedad'


Hallar la media de la puntuación de cada equipo y nombre del equipo ordenada decrecientemente por la media de la puntuación.

In [ ]:
cur=conn.cursor()

cur.execute("SELECT nombre, avg(goles) FROM (SELECT nombre, goles FROM (SELECT nombre, avg(Partidos.goles1) AS goles FROM Equipos join Partidos on Partidos.equipo1=Equipos.nombre GROUP BY nombre ORDER BY goles DESC) UNION SELECT nombre, goles FROM (SELECT nombre, avg(Partidos.goles2) AS goles FROM Equipos join Partidos on Partidos.equipo2=Equipos.nombre GROUP BY nombre ORDER BY goles DESC)) GROUP BY nombre ORDER BY goles DESC")

while True:
    fila=cur.fetchone()
    if not fila: break
    print (fila)
    
cur.close()
conn.commit()

('FC Barcelona', 2.631578947368421)
('Real Madrid', 2.736842105263158)
('Atletico Madrid', 2.026315789473684)
('Athletic Club Bilbao', 1.736842105263158)
('Sevilla FC', 1.8157894736842106)
('Real Sociedad', 1.631578947368421)
('Villarreal CF', 1.5789473684210527)
('Valencia CF', 1.3421052631578947)
('Rayo Vallecano', 1.2105263157894737)
('RC Celta Vigo', 1.2894736842105263)
('UD Almeria', 1.131578947368421)
('Malaga CF', 1.026315789473684)
('RCD Espanyol', 1.0789473684210527)
('CA Osasuna', 0.8421052631578947)
('Getafe CF', 0.9210526315789473)
('Real Valladolid CF', 1.0)
('Levante UD', 0.9210526315789473)
('Real Betis', 0.9473684210526316)
('Granada CF', 0.8421052631578947)
('Elche CF', 0.7894736842105263)
